# Key Data & Analytics Technologies

| **Category** | **Open Source** | **AWS** | **Azure** | **Google Cloud** | **Commercial** |
|--------------|-----------------|---------|-----------|------------------|----------------|
| **Data Ingestion** | Apache Kafka, Apache NiFi | Kinesis, DMS | Event Hubs, Data Factory | Pub/Sub, Dataflow | MuleSoft, Informatica |
| **Storage** | HDFS, Apache Cassandra | S3, Redshift, DynamoDB | Data Lake, Synapse, Cosmos DB | Cloud Storage, BigQuery | Snowflake, MongoDB |
| **Stream Processing** | Apache Flink, Kafka Streams | Kinesis Analytics | Stream Analytics | Dataflow | - |
| **Batch Processing** | Apache Spark, Hadoop | EMR, Glue | HDInsight, Databricks | Dataproc, Dataflow | Databricks |
| **Machine Learning** | TensorFlow, PyTorch, MLflow | SageMaker, Bedrock | Azure ML | AI Platform, Vertex AI | - |
| **Visualization** | Apache Superset, Grafana | QuickSight | Power BI | Looker Studio | Tableau, Qlik |
| **Data Governance** | Apache Atlas, Apache Ranger | Glue Catalog, IAM | Purview, Active Directory | Data Catalog, IAM | Collibra, Immuta |
| **Orchestration** | Apache Airflow, Prefect | Step Functions | Logic Apps | Cloud Composer | - |
| **Monitoring** | Prometheus, Grafana | CloudWatch | Monitor | Cloud Monitoring | Datadog, New Relic |

## Key Technology Pairs by Use Case

| **Use Case** | **Primary Technologies** |
|--------------|-------------------------|
| **Real-time Analytics** | Kafka + Flink + Druid/ClickHouse |
| **Data Lake Architecture** | S3/HDFS + Spark + Airflow + Glue/Atlas |
| **ML Pipeline** | Spark + TensorFlow/PyTorch + MLflow + SageMaker |
| **BI Dashboard** | BigQuery/Snowflake + dbt + Tableau/Power BI |
| **Event-driven Architecture** | Kafka + Flink + Cassandra/DynamoDB |

---
---
## DataBricks :
- General method:
```
Data Sources → Kafka → HDFS → Spark → ETL → Data Warehouse → BI Tools
                    ↓
            ML Platform → Model Serving
```
- Data bricks :
```
Data Sources → Databricks Lakehouse → All Analytics & AI
```

## web topcis to big data :

**Real-Time Messaging:**
- **WebSockets ↔ Kafka:** Both handle real-time, bidirectional communication
- **Server-Sent Events ↔ Data Streams:** Push updates from server to clients

**Performance Optimization:**
- **CDN ↔ Redis/Caching:** Distribute content/data closer to consumers
- **Load Balancers ↔ Data Partitioning:** Spread traffic/data across multiple nodes

**Architecture Patterns:**
- **Microservices ↔ Data Mesh:** Break systems into independent, manageable services
- **Event-Driven Web Apps ↔ Stream Processing:** React to events in real-time

**Communication Patterns:**
- **REST APIs ↔ Data APIs:** Request-response patterns for getting information
- **Pub/Sub (WebSocket rooms) ↔ Kafka Topics:** Broadcast messages to subscribers
- **Webhooks ↔ Change Data Capture:** Notify when something changes

The core insight is that both domains solve the same fundamental challenges: moving information efficiently, handling scale, maintaining reliability, and providing real-time experiences. The solutions are remarkably similar - just applied to different use cases!

# 1. Distributed Storage Systems - Notes

## HDFS (Hadoop Distributed File System)

### What it is:
- Distributed file system (not a traditional database)
- Foundational component for many distributed data systems

### Key Characteristics:
- **Distributed storage** - spans across commodity hardware
- **Fault tolerant** - automatic replication (typically 3x)
- **Large file optimization** - designed for big files & sequential reads
- **Write-once, read-many** - access pattern
- **Block-based** - default 128MB block size

### Systems that use HDFS as storage layer:
- **Apache Hive** → SQL-like queries over HDFS
- **Apache HBase** → NoSQL database on HDFS
- **Apache Spark** → can read/write to HDFS
- **Apache Impala** → real-time SQL queries on HDFS data

### Similar Distributed File Systems:
- **Amazon S3** → object storage service
- **Google Cloud Storage** → object storage
- **Apache Ceph** → unified storage system
- **GlusterFS** → scale-out network-attached storage
- **MinIO** → S3-compatible object storage

### Key Point:
HDFS = distributed storage layer that many distributed databases and analytics systems depend on

*Note: While not a database itself, HDFS is crucial infrastructure in the distributed data ecosystem*


## Hadoop 1.x (2011-2012)
- **Core:** HDFS + MapReduce only
- **Architecture:** MapReduce handles both processing AND resource management
- **Limitation:** Single point of failure (NameNode), limited to batch processing only

## Hadoop 2.x (2013-2017) 
- **Major Addition:** YARN (Yet Another Resource Negotiator) introduced
- **Key Change:** Separated resource management from processing - YARN manages resources, MapReduce just processes
- **Benefits:** Multiple applications can run simultaneously, better resource utilization, NameNode HA added

## Hadoop 3.x (2017-present)
- **Storage:** Erasure coding reduces storage overhead from 200% to 50%
- **Performance:** Better resource utilization, improved MapReduce performance
- **Features:** Multiple NameNodes support, GPU scheduling, enhanced security, Java 8+ requirement

**Key Evolution:** Hadoop 1 → 2 added YARN for multi-tenancy | Hadoop 2 → 3 improved efficiency and reliability